In [1]:
import pandas as pd
import plotly.graph_objects as go
from datetime import time
import yfinance as yf

# Load your Excel data
file_path = 'Todays Python Workflow.xlsx'  # Replace with your path
df = pd.read_excel(file_path)

# Filter out rows where Qty equals 0
df = df[df['Qty'] != 0]

# Ensure the Premium is calculated correctly (Qty * Price * 100)
df['Premium'] = df['Qty'] * df['Price'] * 100

# Sort the data by Expiration to determine "today" and "tomorrow"
df_sorted = df.sort_values(by='Expiration').reset_index(drop=True)

# Identify the first two unique expiration dates (today and tomorrow)
unique_expirations = df_sorted['Expiration'].dropna().unique()

if len(unique_expirations) >= 2:
    today_expiration = unique_expirations[0]  # "Today" is the first date
    tomorrow_expiration = unique_expirations[1]  # "Tomorrow" is the second date
else:
    today_expiration = unique_expirations[0] if len(unique_expirations) == 1 else None
    tomorrow_expiration = None

# Filter the data for 0DTE (today) and 1DTE (tomorrow)
df_0dte_1dte = df_sorted[(df_sorted['Expiration'] == today_expiration) | (df_sorted['Expiration'] == tomorrow_expiration)]

# Clean up Bot/Sold column by stripping extra spaces and converting to uppercase using .loc[]
df_0dte_1dte.loc[:, 'Bot/Sold'] = df_0dte_1dte['Bot/Sold'].str.strip().str.upper()

# Filter based on "Spread" in the Condition column
def filter_spread(df, include_spread=True):
    if include_spread:
        return df  # No filtering, include all transactions
    else:
        return df[df['Condition'] != 'Spread']  # Exclude "Spread" transactions

# Call this function with 'include_spread' set to True or False depending on user preference
include_spread = True  # Change this to False to exclude "Spread" transactions
df_filtered = filter_spread(df_0dte_1dte, include_spread)

# Separate the data for graph (excluding 'MID')
df_graph = df_filtered[df_filtered['Bot/Sold'].isin(['BOT', 'SOLD'])].copy()

# Use Strike Price from column K (renamed for clarity)
df_graph['Strike Price'] = df_graph['Strike']

# Function to handle time parsing (adding leading zeros if necessary)
def parse_time_str(time_str):
    # Ensure time string has leading zeros (e.g., '5:00:00' -> '05:00:00')
    time_str = time_str.zfill(8)
    try:
        return pd.to_datetime(time_str, format='%H:%M:%S', errors='coerce').time()
    except Exception:
        return None

# Apply the custom time parsing function
df_graph['Time_only'] = df_graph['Time'].astype(str).apply(parse_time_str)

# Function to classify trades and count transactions
def classify_trades_with_count(dataframe):
    dataframe['Bot Call'] = dataframe.apply(lambda row: row['Premium'] if row['Type'] == 'Call' and row['Bot/Sold'] == 'BOT' else 0, axis=1)
    dataframe['Sold Call'] = dataframe.apply(lambda row: row['Premium'] if row['Type'] == 'Call' and row['Bot/Sold'] == 'SOLD' else 0, axis=1)
    dataframe['Bot Put'] = dataframe.apply(lambda row: row['Premium'] if row['Type'] == 'Put' and row['Bot/Sold'] == 'BOT' else 0, axis=1)
    dataframe['Sold Put'] = dataframe.apply(lambda row: row['Premium'] if row['Type'] == 'Put' and row['Bot/Sold'] == 'SOLD' else 0, axis=1)
    
    dataframe['Transaction Count'] = 1  # Every row is a transaction
    return dataframe

# Apply classification and transaction count
df_graph = classify_trades_with_count(df_graph)

# Retrieve SPX Spot Price using yfinance
spx_data = yf.Ticker('^GSPC')
spx_spot_price = spx_data.history(period="1d")['Close'].iloc[-1]

# Function to create and show bar charts for different time segments
def create_bar_chart(df_segment, title):
    grouped_data = df_segment.groupby(['Option', 'Strike Price']).agg({
        'Bot Call': 'sum',
        'Sold Call': 'sum',
        'Bot Put': 'sum',
        'Sold Put': 'sum',
        'Transaction Count': 'sum'  # Sum up transactions
    }).reset_index()

    # Sort and select top 20 by total premium
    grouped_data['Total Premium'] = grouped_data['Bot Call'] + grouped_data['Sold Call'] + grouped_data['Bot Put'] + grouped_data['Sold Put']
    top_20_strikes = grouped_data.sort_values(by='Total Premium', ascending=False).head(20)['Strike Price'].values

    # Filter original data to keep only the top 20 strike prices
    df_segment_top_20 = df_segment[df_segment['Strike Price'].isin(top_20_strikes)].sort_values(by=['Strike Price', 'Premium'])

    fig = go.Figure()

    # Add bars for Bot Calls (each individual transaction as its own bar)
    bot_call_transactions = df_segment_top_20[(df_segment_top_20['Bot Call'] > 0)]
    fig.add_trace(go.Bar(
        x=bot_call_transactions['Strike Price'], 
        y=bot_call_transactions['Bot Call'], 
        name='Bot Call', 
        marker_color='blue',
        hovertemplate=(
            'Premium: %{y:$,.2f}<br>' +
            'Time: %{customdata[0]}'
        ),
        customdata=bot_call_transactions[['Time']].values  # Pass the transaction time
    ))

    # Add bars for Sold Calls (each individual transaction as its own bar)
    sold_call_transactions = df_segment_top_20[(df_segment_top_20['Sold Call'] > 0)]
    fig.add_trace(go.Bar(
        x=sold_call_transactions['Strike Price'], 
        y=sold_call_transactions['Sold Call'], 
        name='Sold Call', 
        marker_color='lightcoral',
        hovertemplate=(
            'Premium: %{y:$,.2f}<br>' +
            'Time: %{customdata[0]}'
        ),
        customdata=sold_call_transactions[['Time']].values  # Pass the transaction time
    ))

    # Add bars for Bot Puts (each individual transaction as its own bar)
    bot_put_transactions = df_segment_top_20[(df_segment_top_20['Bot Put'] > 0)]
    fig.add_trace(go.Bar(
        x=bot_put_transactions['Strike Price'], 
        y=bot_put_transactions['Bot Put'], 
        name='Bot Put', 
        marker_color='red',
        hovertemplate=(
            'Premium: %{y:$,.2f}<br>' +
            'Time: %{customdata[0]}'
        ),
        customdata=bot_put_transactions[['Time']].values  # Pass the transaction time
    ))

    # Add bars for Sold Puts (each individual transaction as its own bar)
    sold_put_transactions = df_segment_top_20[(df_segment_top_20['Sold Put'] > 0)]
    fig.add_trace(go.Bar(
        x=sold_put_transactions['Strike Price'], 
        y=sold_put_transactions['Sold Put'], 
        name='Sold Put', 
        marker_color='lightblue',
        hovertemplate=(
            'Premium: %{y:$,.2f}<br>' +
            'Time: %{customdata[0]}'
        ),
        customdata=sold_put_transactions[['Time']].values  # Pass the transaction time
    ))

    # Add a vertical dashed yellow line for SPX Spot Price
    fig.add_vline(
        x=spx_spot_price, 
        line=dict(color='yellow', dash='dash'),
        annotation_text=f'SPX Spot: {spx_spot_price:.2f}', 
        annotation_position="top"
    )

    # Set X-axis to show all Strike Prices in the top 20
    fig.update_layout(
        xaxis=dict(
            title='Strike Price',
            tickmode='array',
            tickvals=df_segment_top_20['Strike Price'].unique(),  # Show only top 20 strike prices
            ticktext=df_segment_top_20['Strike Price'].unique().astype(str),
            range=[df_segment_top_20['Strike Price'].min(), df_segment_top_20['Strike Price'].max()]
        ),
        yaxis_title='Premium Amount',
        title=title,
        barmode='stack',  # Stack bars to visually segment by transactions
        template='plotly_dark'
    )

    fig.show()

# Define new hourly time segments (5 a.m. to 2 p.m.)
time_segments = [
    (time(5, 0, 0), time(6, 0, 0)),
    (time(6, 0, 0), time(7, 0, 0)),
    (time(7, 0, 0), time(8, 0, 0)),
    (time(8, 0, 0), time(9, 0, 0)),
    (time(9, 0, 0), time(10, 0, 0)),
    (time(10, 0, 0), time(11, 0, 0)),
    (time(11, 0, 0), time(12, 0, 0)),
    (time(12, 0, 0), time(1, 0, 0)),
    (time(1, 0, 0), time(2, 0, 0))
]

# Create charts for each hourly time segment
for start_time, end_time in time_segments:
    df_segment = df_graph[(df_graph['Time_only'] >= start_time) & (df_graph['Time_only'] < end_time)]
    
    # Debugging: Check data for each time segment
    print(f"Data for time segment {start_time.strftime('%H:%M:%S')} - {end_time.strftime('%H:%M:%S')}:")
    print(df_segment[['Time_only', 'Option', 'Strike Price', 'Premium']].head())

    title = f'Premium by Option Chain ({start_time.strftime("%H:%M:%S")}-{end_time.strftime("%H:%M:%S")})'
    create_bar_chart(df_segment, title)


Data for time segment 05:00:00 - 06:00:00:
      Time_only            Option  Strike Price  Premium
20802  05:41:11  10 OCT 24 5645 P          5645     30.0
20803  05:41:09  10 OCT 24 5885 C          5885      5.0
20805  05:41:01  10 OCT 24 5635 P          5635     20.0
20806  05:41:01  10 OCT 24 5660 P          5660     50.0
20807  05:41:01  10 OCT 24 5550 P          5550     10.0


Data for time segment 06:00:00 - 07:00:00:
      Time_only            Option  Strike Price  Premium
15863  06:59:57  10 OCT 24 5625 P          5625     10.0
15866  06:59:57  10 OCT 24 5650 P          5650     15.0
15904  06:59:50  10 OCT 24 5585 P          5585      5.0
15905  06:59:50  10 OCT 24 5660 P          5660     20.0
15906  06:59:50  10 OCT 24 5510 C          5510  26500.0


Data for time segment 07:00:00 - 08:00:00:
      Time_only            Option  Strike Price  Premium
15846  07:00:04  10 OCT 24 5655 P          5655     20.0
15847  07:00:04  10 OCT 24 5655 P          5655     40.0
15850  07:00:03  10 OCT 24 5925 C          5925      5.0
15851  07:00:03  10 OCT 24 5645 P          5645     15.0
15852  07:00:03  10 OCT 24 5925 C          5925      5.0


Data for time segment 08:00:00 - 09:00:00:
      Time_only            Option  Strike Price  Premium
23772  08:54:59  10 OCT 24 5625 P          5625     10.0
23773  08:54:58  10 OCT 24 5840 C          5840     50.0
23779  08:54:48  10 OCT 24 5855 C          5855     15.0
23781  08:55:01  10 OCT 24 5685 P          5685     10.0
23793  08:54:35  10 OCT 24 5650 P          5650     10.0


Data for time segment 09:00:00 - 10:00:00:
     Time_only            Option  Strike Price  Premium
3963  09:32:28  10 OCT 24 5795 P          5795   1120.0
3965  09:32:27  10 OCT 24 5705 P          5705     80.0
3966  09:32:27  10 OCT 24 5705 P          5705     40.0
3967  09:32:27  10 OCT 24 5705 P          5705     40.0
3968  09:32:27  10 OCT 24 5705 P          5705     20.0


Data for time segment 10:00:00 - 11:00:00:
   Time_only            Option  Strike Price  Premium
7   10:12:25  10 OCT 24 5600 P          5600      5.0
8   10:12:24  10 OCT 24 5700 P          5700     20.0
9   10:12:21  10 OCT 24 5710 P          5710     25.0
11  10:12:20  10 OCT 24 5690 P          5690     15.0
12  10:12:19  10 OCT 24 5800 P          5800   2480.0


Data for time segment 11:00:00 - 12:00:00:
     Time_only            Option  Strike Price  Premium
7923  11:11:01  10 OCT 24 5805 P          5805   2850.0
7924  11:11:01  10 OCT 24 5830 C          5830     40.0
7925  11:11:01  10 OCT 24 5830 C          5830      5.0
7926  11:11:01  10 OCT 24 5830 C          5830      5.0
7927  11:11:01  10 OCT 24 5830 C          5830     10.0


Data for time segment 12:00:00 - 01:00:00:
Empty DataFrame
Columns: [Time_only, Option, Strike Price, Premium]
Index: []


Data for time segment 01:00:00 - 02:00:00:
Empty DataFrame
Columns: [Time_only, Option, Strike Price, Premium]
Index: []
